In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ngl import *
from tqdm import tqdm

In [ ]:
def icosphere(level):
    """
    Build an R-refined icosahedral mesh on the unit sphere.

    level = 0 -> base icosahedron (12 vertices, 20 faces)
    level = 1 -> 42 vertices, 80 faces
    ...
    level = 6 -> 40962 vertices, 81920 faces  (GraphCast-like)
    """
    def _create_icosahedron():
        """Regular unit icosahedron (12 vertices, 20 faces) on the unit sphere."""
        phi = (1.0 + np.sqrt(5.0)) / 2.0

        # Raw coordinates
        verts = np.array([
            [-1,  phi,  0],
            [ 1,  phi,  0],
            [-1, -phi,  0],
            [ 1, -phi,  0],
            [ 0, -1,  phi],
            [ 0,  1,  phi],
            [ 0, -1, -phi],
            [ 0,  1, -phi],
            [ phi,  0, -1],
            [ phi,  0,  1],
            [-phi,  0, -1],
            [-phi,  0,  1],
        ], dtype=float)

        # Normalize to unit sphere
        verts /= np.linalg.norm(verts, axis=1, keepdims=True)

        # Faces (triangles) — standard icosahedron connectivity
        faces = np.array([
            [0, 11, 5],
            [0, 5, 1],
            [0, 1, 7],
            [0, 7, 10],
            [0, 10, 11],
            [1, 5, 9],
            [5, 11, 4],
            [11, 10, 2],
            [10, 7, 6],
            [7, 1, 8],
            [3, 9, 4],
            [3, 4, 2],
            [3, 2, 6],
            [3, 6, 8],
            [3, 8, 9],
            [4, 9, 5],
            [2, 4, 11],
            [6, 2, 10],
            [8, 6, 7],
            [9, 8, 1],
        ], dtype=int)

        return verts, faces
    def _subdivide_icosphere(vertices, faces):
        """
        One refinement step: split each triangle into 4.
        Vertices stay on the unit sphere.
        """
        # We'll add new vertices on the fly and cache midpoints so we don't duplicate them.
        midpoint_cache = {}
        verts_list = vertices.tolist()
        new_faces = []

        def get_midpoint(i, j):
            key = tuple(sorted((i, j)))
            if key in midpoint_cache:
                return midpoint_cache[key]

            vi = np.array(verts_list[i])
            vj = np.array(verts_list[j])
            m = (vi + vj) * 0.5
            m /= np.linalg.norm(m)  # project to unit sphere

            verts_list.append(m.tolist())
            idx = len(verts_list) - 1
            midpoint_cache[key] = idx
            return idx

        for tri in faces:
            i, j, k = tri
            a = get_midpoint(i, j)
            b = get_midpoint(j, k)
            c = get_midpoint(k, i)

            # 4 new triangles
            new_faces.append([i, a, c])
            new_faces.append([j, b, a])
            new_faces.append([k, c, b])
            new_faces.append([a, b, c])

        new_vertices = np.array(verts_list, dtype=float)
        new_faces = np.array(new_faces, dtype=int)
        return new_vertices, new_faces
    verts, faces = _create_icosahedron()
    for _ in range(level):
        verts, faces = _subdivide_icosphere(verts, faces)
    return verts, faces

In [ ]:
icos_verts, icos_faces = icosphere(level=5)
icos_lats = np.degrees(np.arcsin(icos_verts[:, 2]))
icos_lons = np.degrees(np.arctan2(icos_verts[:, 1], icos_verts[:, 0]))

In [ ]:
stations = fetch_station_24h_final()
stations['x'] = np.cos(np.radians(stations['lat'])) * np.cos(np.radians(stations['lon']))
stations['y'] = np.cos(np.radians(stations['lat'])) * np.sin(np.radians(stations['lon']))
stations['z'] = np.sin(np.radians(stations['lat']))
stations_lat_lon = stations[['lat', 'lon']].values

In [ ]:
from sklearn.neighbors import BallTree
from sklearn.metrics.pairwise import haversine_distances    

In [ ]:
stations_balltree = BallTree(stations_lat_lon,  metric='haversine')

In [ ]:
for station_name in tqdm(stations['station'].values):
    try:
        fetch_IGS20_24h_final(station_name, "tenv", "IGS20_24h_final/", overwrite=False)
    except Exception as e:
        tqdm.write(f"Failed to fetch data for station {station_name}: {e}")
        continue
    

In [ ]:
dataframes = []
for station_file in tqdm(list(pathlib.Path("IGS20_24h_final/").glob("*.csv"))):
    try:
        dataframes.append(pd.read_csv(station_file.absolute(), sep=",", parse_dates=['date']))
    except Exception as e:
        tqdm.write(f"Failed to read data for station {station_file.name}: {e}")
        continue
dataframes = pd.concat(dataframes, ignore_index=True)
dataframes['date'] = dataframes['date'].values.astype('datetime64[D]')

In [ ]:
#dataframes.set_index(['station', 'date'], inplace=True)

In [ ]:
unique_dates = np.unique(dataframes['date']).astype('datetime64[D]')
min_date = unique_dates.min()
max_date = unique_dates.max()

In [ ]:
cols = ['delta_e_m','delta_n_m','delta_v_m']

df = dataframes[['station', 'date'] + cols].copy()
df['date'] = pd.to_datetime(df['date'])
df = df.sort_values(['station','date'])

g = df.groupby('station', sort=False)

# raw diffs
df[cols] = g[cols].diff()

# only keep diffs if previous row is exactly 1 day earlier
dt = g['date'].diff()
ok = dt.eq(pd.Timedelta(days=1))

df.loc[~ok, cols] = np.nan   # breaks at gaps (and first row per station)


In [ ]:
window_size_num_days = 7
window_size = np.timedelta64(window_size_num_days, 'D')
cols = ['delta_e_m', 'delta_n_m', 'delta_v_m']
nan_row = pd.Series({c: np.nan for c in cols})
def displacement_extract(df):
    start_pos = df.iloc[0][cols]
    end_pos = df.iloc[-1][cols]
    displacement = end_pos - start_pos
    return displacement
all_displacements = []
for start_date in tqdm(np.arange(min_date, max_date, np.timedelta64(1, 'D'))):
    subset_mask = (dataframes['date'] >= start_date) & (dataframes['date'] < start_date + window_size)
    subset = dataframes.loc[subset_mask]
    subset = subset.groupby('station').filter(lambda x: len(x) == window_size_num_days).reset_index(drop=True)
    displacements = subset[['station', 'date'] + cols].groupby('station').apply(lambda df: displacement_extract(df.sort_values('date')), include_groups=False)
    displacements['start_date'] = start_date
    displacements['end_date'] = start_date + window_size - np.timedelta64(1, 'D')
    all_displacements.append(displacements)
all_displacements = pd.concat(all_displacements, ignore_index=True)